## Setting up exponential back off and OpenAI API

In [1]:
# !pip install langchain openai weaviate-client python-dotenv tiktoken tenacity

In [2]:
# Setting exponential back off
# source: https://tenacity.readthedocs.io/en/latest/ 

import tenacity 
from tenacity import *
from tenacity import retry

@retry(wait=wait_exponential(multiplier=1, min=4, max=10))
def wait_exponential_1():
    print("Wait 2^x * 1 second between each retry starting with 4 seconds, then up to 10 seconds, then 10 seconds afterwards")
    raise Exception

In [3]:
#!pip install virtualenv

In [4]:
# Importing OpenAI API key
import os
import openai
import sys
from openai import OpenAI

openai.api_key  = os.environ['OPENAI_API_KEY']

In [5]:
# Loading environment
import dotenv
from dotenv import load_dotenv

dotenv.load_dotenv()

False

In [6]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

## Importing data

In [7]:
#!pip install pypdf

In [8]:
# Imorting data
from langchain.document_loaders import PyPDFLoader

data_path = "/Users/teamihajlov/EELM_Application/texts_final.pdf"
loader = PyPDFLoader(data_path)
pages = loader.load()

In [9]:
len(pages)

97

In [10]:
type(pages)

list

In [11]:
page = pages[0]
print(pages[0])

page_content='thats how it all began scientists believe around 1377 billion years ago our own part as geologists in this ongoing story began much more recently with the systematization of the scientific study of the earth in the 18th century earth science began to investigate the air we breathe and the waters that surround our continents the land itself the rocks minerals soils the geographical features such as mountain ranges and deserts the living creatures with which we share our planet every aspect of the earth earth scientists like their colleagues in other fields have developed a scientific method based on observation and collection of data formulation and testing of hypotheses leading to the formulation of laws or principles this study expanded in the 1950s as humanity began to explore space one aspect of this expansion has been that earth could now be studied from a new perspective u carried into orbit by satellites this has resulted in an increased ability to test and prove hy

In [12]:
page.metadata

{'source': '/Users/teamihajlov/EELM_Application/texts_final.pdf', 'page': 0}

### Splitting the documents

In [13]:
# Setting up chunk size and overlap
chunk_size = 512
chunk_overlap = 50

In [14]:
# Trying out RecursiveCharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    length_function=len
)

chunks = text_splitter.split_documents(pages)
print(chunks[0])

page_content='thats how it all began scientists believe around 1377 billion years ago our own part as geologists in this ongoing story began much more recently with the systematization of the scientific study of the earth in the 18th century earth science began to investigate the air we breathe and the waters that surround our continents the land itself the rocks minerals soils the geographical features such as mountain ranges and deserts the living creatures with which we share our planet every aspect of the earth earth' metadata={'source': '/Users/teamihajlov/EELM_Application/texts_final.pdf', 'page': 0}


In [15]:
len(chunks)

773

In [16]:
# Creating embeddings
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Weaviate
import weaviate
from weaviate.embedded import EmbeddedOptions

client = weaviate.Client(
  embedded_options = EmbeddedOptions()
)

vectorstore = Weaviate.from_documents(
    client = client,
    documents = chunks,
    embedding = OpenAIEmbeddings(),
    by_text = False
)

/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/weaviate/warnings.py:158: DeprecationWarning: Dep016: You are using the Weaviate v3 client, which is deprecated.
            Consider upgrading to the new and improved v4 client instead!
            See here for usage: https://weaviate.io/developers/weaviate/client-libraries/python
            
  warnings.warn(
{"action":"startup","default_vectorizer_module":"none","level":"info","msg":"the default vectorizer modules is set to \"none\", as a result all new schema classes without an explicit vectorizer setting, will use this vectorizer","time":"2024-03-28T21:55:02+01:00"}
{"action":"startup","auto_schema_enabled":true,"level":"info","msg":"auto schema enabled setting is set to \"true\"","time":"2024-03-28T21:55:02+01:00"}
{"level":"info","msg":"No resource limits set, weaviate will use all available memory and CPU. To limit resources, set LIMIT_RESOURCES=true","time":"2024-03-28T21:55:02+01:00"}


Started /Users/teamihajlov/.cache/weaviate-embedded: process ID 3461


{"level":"warning","msg":"Multiple vector spaces are present, GraphQL Explore and REST API list objects endpoint module include params has been disabled as a result.","time":"2024-03-28T21:55:02+01:00"}
{"action":"grpc_startup","level":"info","msg":"grpc server listening at [::]:50060","time":"2024-03-28T21:55:02+01:00"}
{"action":"restapi_management","level":"info","msg":"Serving weaviate at http://127.0.0.1:8079","time":"2024-03-28T21:55:02+01:00"}
/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
{"level":"info","msg":"Created shard langchain_f

## Importing queries

In [17]:
import pandas as pd

path_queries = "/Users/teamihajlov/EELM_Application/EELM_queries_golden.csv"
queries = pd.read_csv(path_queries, encoding= "utf-8")
queries.head()

,Question,Golden
0,What is biosphere?,Biosphere is one of the layers of the Earth. I...
1,What is fossilisation?,Fossilisation is the process of parts of a dea...
2,"In the context of geology and rock formaiton, ...",Degradation the process in which a rock is bei...
3,What is metamorphism?,Metamorphism is the process in which rock’s ch...
4,What is chemical weathering?,Chemical weathering implies transformation of ...


In [18]:
queries.shape

(49, 2)

In [19]:
queries_list = []

for query in queries["Question"]:
    queries_list.append(query)

print(queries_list)
print(len(queries_list))

['What is biosphere?', 'What is fossilisation?', 'In the context of geology and rock formaiton, what is degradation?', 'What is metamorphism?', 'What is chemical weathering?', 'What is sporopollenin?', 'What is detritus?', 'What is topography?', 'What is beryl?', 'What is a metamorphic rock?', 'What are fossil fuels?', 'What is a mineral district?', 'What is till?', 'Explain what global warming is and why it happens.', 'What is an igneous rock?', 'What is erosion?', 'What is a gemstone?', 'What is a glacier?', 'In the context of geology, what is inclusion?', 'What is a hypothesis?', 'What is groundwater?', 'What is a debris?', 'What is bedrock?', 'What is weathering?', 'What is a zircon?', 'What is subsidence?', 'What is exploitation?', 'In the context of geology, what is a fault?', 'What is lustre?', 'In the context of geology, what is a cavity?', 'In the context of geology, what is suspension?', 'What is a borehole?', 'What is a coastline?', 'What is foliation?', 'What is a microfoss

### Similarity Search

In [20]:
question = queries_list[3]
vectorstore.similarity_search(question, k=3)

/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


[Document(page_content='of superhot magma contact metamorphism into any form of rock can trigger metamorphic changes through thermic and chemical reactions as can the physical pressure of tectonic plate movements and collisions regional metamorphism thereby forming new metamorphic rock and mineral deposits gemstones associated with metamorphism which are frequently highly durable include ruby sapphire jade lapis lazuli and once again zircon variations there are of course other significant factors besides the phases of the rock', metadata={'page': 43, 'source': '/Users/teamihajlov/EELM_Application/texts_final.pdf'}),
 Document(page_content='in the way different minerals crystallise out of molten lava at specific temperatures in the formation of igneous rocks the same process occurs in metamorphism only in what is termed solid state alteration temperature and pressure cause the minerals to recrystallise in new forms although the rock remains solid throughout this process the newly formed

In [21]:
vectorstore.max_marginal_relevance_search(question,k=3)

[Document(page_content='of superhot magma contact metamorphism into any form of rock can trigger metamorphic changes through thermic and chemical reactions as can the physical pressure of tectonic plate movements and collisions regional metamorphism thereby forming new metamorphic rock and mineral deposits gemstones associated with metamorphism which are frequently highly durable include ruby sapphire jade lapis lazuli and once again zircon variations there are of course other significant factors besides the phases of the rock', metadata={'page': 43, 'source': '/Users/teamihajlov/EELM_Application/texts_final.pdf'}),
 Document(page_content='transformation of large deposits into smaller pieces of rock and minerals and on into soils this stage consists of two interlinked but discrete processes weathering and erosion on the surface of the lithosphere rock is weathered or exposed to the actions of air wind water the biosphere and other rocks themselves which tends to break the rock down int

## Vectorstore retrieval

In [22]:
# Setting up a retriever
retriever = vectorstore.as_retriever(search_kwargs = {'k': 5})

In [23]:
# Creating a prompt template
from langchain.prompts import ChatPromptTemplate

template = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use three sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""
prompt = ChatPromptTemplate.from_template(template)

print(prompt)

input_variables=['context', 'question'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks.\nUse the following pieces of retrieved context to answer the question.\nIf you don't know the answer, just say that you don't know.\nUse three sentences maximum and keep the answer concise.\nQuestion: {question}\nContext: {context}\nAnswer:\n"))]


### Creating a data frame for results

In [24]:
# Defining a function for storing results

def create_results(chain, questions):
    df = pd.DataFrame()
    answers = []
    context = []
    for question in questions:
        response = chain.invoke(question)
        answers.append(response['result'])
        subcontext = []
        for doc in response['source_documents']:
            
            subcontext.append(doc.page_content)
        
        context.append(subcontext)
        
    df["Question"] = questions
    df["Answer"] = answers
    df["Context"] = context  
        
        
    return(df)

## Retrieval with no additional parameters

In [25]:
# Creating a chain

from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.chains import RetrievalQA

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}
)

/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [26]:
queries_mini_list = []
for query in queries["Question"][0:5]:
    queries_mini_list.append(query)

print(queries_mini_list)

['What is biosphere?', 'What is fossilisation?', 'In the context of geology and rock formaiton, what is degradation?', 'What is metamorphism?', 'What is chemical weathering?']


In [27]:
# Testing the create_results function
res_mini = create_results(qa_chain, queries_mini_list)

/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in

In [28]:
type(res_mini['Context'][0])

list

In [29]:
res_mini.to_pickle('mini_pickle.pkl')

In [30]:
results_qa_chain = create_results(qa_chain, queries_list)

/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in

/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in

/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in

/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in

/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in

In [31]:
results_qa_chain.head()

,Question,Answer,Context
0,What is biosphere?,The biosphere is a layer of the earth that inc...,[further cycles within them biogeochemical cyc...
1,What is fossilisation?,Fossilisation is the process where sediments f...,[sediments often on the bed of a lake or sea t...
2,"In the context of geology and rock formaiton, ...",Degradation in geology refers to the process o...,[constituent partsdecay degradation disintegra...
3,What is metamorphism?,Metamorphism is the process where rocks underg...,[of superhot magma contact metamorphism into a...
4,What is chemical weathering?,Chemical weathering is the transformation of r...,[area which is where the reactions of chemical...


In [32]:
results_qa_chain.shape

(49, 3)

In [33]:
type(results_qa_chain['Context'][0])

list

In [34]:
results_qa_chain['Context'][0]

['further cycles within them biogeochemical cycles represent an interesting example here charting the pathway of chemical substances such as carbon hydrogen and oxygen through both the biotic biosphere and abiotic lithosphere hydrosphere atmosphere segments of the physical world while the processes and cycles occurring on and below the lithosphere tend to move very slowly and are often imperceptible the external processes those in the atmosphere hydrosphere and biosphere have a tendency to be more rapid and',
 'biosphere have a tendency to be more rapid and more easily observed the encounter of these slower and faster processes on the earths surface contributes to the rock cycle through the continuous degradation and erosion of rocks there is a tendency to regard recycling as a modern phenomenon an attempt by the green minded to save the earth from drowning in a sea of discarded paper plastic and glass in point of fact the earth was millions of years ahead of us for the rock cycle can 

## Adding Maximum marginal relevance (MMR)

In [35]:
qa_mmr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectorstore.as_retriever(search_kwargs = {'k': 5}, search_type = "mmr"),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}
)

In [36]:
# Testing the retrieval
for question in queries_mini_list:
    print(f"Question: {question}")
    print(f"Answer: {qa_mmr.invoke(question)}")
    print()

Question: What is biosphere?


/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


Answer: {'query': 'What is biosphere?', 'result': 'The biosphere is a layer of the earth that includes all life forms. It encompasses all living organisms on Earth, from plants to animals. The biosphere plays a crucial role in supporting life on our planet.', 'source_documents': [Document(page_content='further cycles within them biogeochemical cycles represent an interesting example here charting the pathway of chemical substances such as carbon hydrogen and oxygen through both the biotic biosphere and abiotic lithosphere hydrosphere atmosphere segments of the physical world while the processes and cycles occurring on and below the lithosphere tend to move very slowly and are often imperceptible the external processes those in the atmosphere hydrosphere and biosphere have a tendency to be more rapid and', metadata={'page': 22, 'source': '/Users/teamihajlov/EELM_Application/texts_final.pdf'}), Document(page_content='a planetbelieve an action of considering something to be truethink supp

/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


Answer: {'query': 'What is fossilisation?', 'result': 'Fossilisation is the process of parts of a dead animal or plant being turned into a part of sediment and preserved. Fossilised remains are parts of animals or plants that have been hardened and preserved in sediment. The process often preserves only the harder parts of an organism, leading to gaps in what may be discovered.', 'source_documents': [Document(page_content='sediments often on the bed of a lake or sea that will one day form sedimentary rock a fossil is formed often reinforced by the accretion of minerals in the sediment in some cases an encased corpse may dissolve leaving a hollow imprint of the body known as a mould which may fill up with minerals which then solidify in the same form producing a cast the carbon filaments and films of leaves can be fossilised as can the burrows left by an animal its tracks even its droppings such traces and records of earlier', metadata={'page': 14, 'source': '/Users/teamihajlov/EELM_App

...

/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


Answer: {'query': 'What is metamorphism?', 'result': 'Metamorphism is the process in which the chemical, mineralogical, and/or textural composition of a rock changes due to alteration. Heat, pressure, and chemically active fluids are the main agents involved in metamorphism. It occurs at temperatures above 200°C and with associated pressure above 300 megapascals.', 'source_documents': [Document(page_content='of superhot magma contact metamorphism into any form of rock can trigger metamorphic changes through thermic and chemical reactions as can the physical pressure of tectonic plate movements and collisions regional metamorphism thereby forming new metamorphic rock and mineral deposits gemstones associated with metamorphism which are frequently highly durable include ruby sapphire jade lapis lazuli and once again zircon variations there are of course other significant factors besides the phases of the rock', metadata={'page': 43, 'source': '/Users/teamihajlov/EELM_Application/texts_fi

/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


Answer: {'query': 'What is chemical weathering?', 'result': 'Chemical weathering occurs when the chemical structure of minerals is broken down or changed through reactions when rock is exposed to air or acidic water. It is a process that accelerates in areas where mechanical weathering is also happening. The distinction between mechanical and chemical weathering is based on the changes in the rock itself.', 'source_documents': [Document(page_content='area which is where the reactions of chemical weathering take place the processes of chemical weathering are accelerated by those of mechanical weathering chemical weathering chemical weathering occurs when the chemical structure of minerals is broken down or changed through a complex process of reactions which occur when rock is exposed to air or more frequently moderately acidic water most often in the form of acid rain while acid rain is often thought of as a consequence of human derived industrial', metadata={'page': 28, 'source': '/Us

/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


In [37]:
results_qa_mmr = create_results(qa_mmr,queries_list)

/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in

/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in

/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in

/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in

/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in

In [38]:
results_qa_mmr.head()

,Question,Answer,Context
0,What is biosphere?,The biosphere is a layer of the earth that inc...,[further cycles within them biogeochemical cyc...
1,What is fossilisation?,Fossilisation is the process of parts of a dea...,[sediments often on the bed of a lake or sea t...
2,"In the context of geology and rock formaiton, ...",Degradation in geology refers to the process o...,[constituent partsdecay degradation disintegra...
3,What is metamorphism?,Metamorphism is the process in which the chemi...,[of superhot magma contact metamorphism into a...
4,What is chemical weathering?,Chemical weathering occurs when the chemical s...,[area which is where the reactions of chemical...


In [39]:
results_qa_mmr.shape

(49, 3)

In [40]:
type(results_qa_mmr['Context'][0])

list

In [41]:
results_qa_mmr['Context'][0]

['further cycles within them biogeochemical cycles represent an interesting example here charting the pathway of chemical substances such as carbon hydrogen and oxygen through both the biotic biosphere and abiotic lithosphere hydrosphere atmosphere segments of the physical world while the processes and cycles occurring on and below the lithosphere tend to move very slowly and are often imperceptible the external processes those in the atmosphere hydrosphere and biosphere have a tendency to be more rapid and',
 'a planetbelieve an action of considering something to be truethink suppose assume presumeberyl a mineral made up of a silicate of beryllium and aluminum colorless when pure and green blue pink or yellow when not purebiosphere a layer of the earth that includes all life formsbond the link between two atoms created by electromagnetic forces that hold the atoms togetherbrittle used for describing something that breaks easilybuild up make something soundbigger or more important than

## Adding Compression

In [42]:
#!pip install flask-sqlalchemy --user
#!pip install pyqt5 --user
#!pip install pyqtwebengine --user

In [43]:
#!conda install sqlalchemy

In [44]:
import sqlalchemy
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))

In [45]:
# Wrap our vectorstore
compressor = LLMChainExtractor.from_llm(llm)

In [46]:
# Creating a compressor
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectorstore.as_retriever(search_kwargs = {'k': 5})
)

In [47]:
# Creating a chain with compression retrieval
qa_compress = RetrievalQA.from_chain_type(
    llm,
    retriever=compression_retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}
)

In [48]:
# Testing the retrieval
create_results(qa_compress,queries_mini_list)

/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', cat

/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', cat

/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', cat

,Question,Answer,Context
0,What is biosphere?,The biosphere is a layer of the earth that inc...,"[biotic biosphere, biosphere a layer of the ea..."
1,What is fossilisation?,Fossilisation is the process of preserving the...,[sediments often on the bed of a lake or sea t...
2,"In the context of geology and rock formaiton, ...",Degradation in geology refers to the process o...,[degradation the process in which a rock is be...
3,What is metamorphism?,Metamorphism is the process where rocks underg...,[contact metamorphism into any form of rock ca...
4,What is chemical weathering?,Chemical weathering is the process where miner...,[chemical weathering occurs when the chemical ...


In [49]:
results_qa_compress = create_results(qa_compress, queries_list)

/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', cat

/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', cat

/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', cat

/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', cat

/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', cat

...

/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', cat

/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in

/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser direct

/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', cat

/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', cat

In [50]:
results_qa_compress.head()

,Question,Answer,Context
0,What is biosphere?,The biosphere is the layer of the earth that i...,"[biotic biosphere, biosphere a layer of the ea..."
1,What is fossilisation?,Fossilisation is the process of preserving the...,[sediments often on the bed of a lake or sea t...
2,"In the context of geology and rock formaiton, ...",Degradation in geology refers to the process o...,[degradation the process in which a rock is be...
3,What is metamorphism?,Metamorphism is the process where minerals rec...,[trigger metamorphic changes through thermic a...
4,What is chemical weathering?,Chemical weathering is the process where rocks...,[chemical weathering occurs when the chemical ...


In [51]:
results_qa_compress.shape

(49, 3)

In [52]:
type(results_qa_compress['Context'][0])

list

In [53]:
results_qa_compress['Context'][0]

['biotic biosphere',
 'biosphere a layer of the earth that includes all life forms',
 'biosphere']

### Saving the results

In [58]:
results_qa_compress.to_pickle('results_qa_compress.pkl')

In [59]:
results_qa_chain.to_pickle('results_qa_chain.pkl')

In [60]:
results_qa_mmr.to_pickle('results_qa_mmr.pkl')